## CS 644 - Final Project 

##### Group Members :- Smeet Kathiria 



# Project Part 1 -> Data Analysis

In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("./training.1600000.processed.noemoticon.csv",header=None, names=['sentiment','id','date','query_string','user','text'], encoding='ISO-8859-1')


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   sentiment     1600000 non-null  int64 
 1   id            1600000 non-null  int64 
 2   date          1600000 non-null  object
 3   query_string  1600000 non-null  object
 4   user          1600000 non-null  object
 5   text          1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
df.drop(['id','date','query_string','user'],axis=1,inplace=True)


In [ ]:
df.head()


sentiment                                               text
0          0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          0  is upset that he can't update his Facebook by ...
2          0  @Kenichan I dived many times for the ball. Man...
3          0    my whole body feels itchy and like its on fire 
4          0  @nationwideclass no, it's not behaving at all....

In [ ]:
df.sentiment.value_counts()


4    800000
0    800000
Name: sentiment, dtype: int64

##### There are total 1.6 million rows and no null values, also there is no sentiment with neutral class only positive and negative values are there. There is no skewness since the data is equally divided amon positve and negative class. 

In [ ]:
df[df.sentiment == 4].head(10)


sentiment                                               text
800000          4       I LOVE @Health4UandPets u guys r the best!! 
800001          4  im meeting up with one of my besties tonight! ...
800002          4  @DaRealSunisaKim Thanks for the Twitter add, S...
800003          4  Being sick can be really cheap when it hurts t...
800004          4    @LovesBrooklyn2 he has that effect on everyone 
800005          4  @ProductOfFear You can tell him that I just bu...
800006          4  @r_keith_hill Thans for your response. Ihad al...
800007          4  @KeepinUpWKris I am so jealous, hope you had a...
800008          4  @tommcfly ah, congrats mr fletcher for finally...
800009          4  @e4VoIP I RESPONDED  Stupid cat is helping me ...

##### looks like all the tweets with positive sentiment starts from 800000 index  
 



In [ ]:
import seaborn as sns


In [ ]:
plt.figure(figsize = (18, 8))
plt.style.use('fivethirtyeight')
ax = sns.countplot('sentiment', data = df, palette = 'dark')
ax.set_xlabel(xlabel = '0: negative, 4: positive', fontsize = 16)
ax.set_ylabel(ylabel = 'Count of sentiment', fontsize = 16)
ax.set_title(label = 'Comparison of sentiments and counts', fontsize = 20)
plt.show()

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


##### later we will convert the all the sentiment classified as 4 to 1 just to be more consistent 




In [ ]:
from wordcloud import WordCloud  # importing the worldcloud module. Wordcloud uses the text sizes to represent the frequency of the text

In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='white',
                          width=1920,
                          height=1080
                         ).generate(" ".join(df.text))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')
plt.show()

##### Checking the length of string in text column  




In [ ]:
df['text_len'] = [len(t) for t in df.text]


In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
df[df.text_len > 160].head(10)


sentiment  ... text_len
13538           0  ...      167
16809           0  ...      161
19818           0  ...      168
26880           0  ...      172
29196           0  ...      186
50881           0  ...      166
68913           0  ...      161
77884           0  ...      166
99690           0  ...      173
106045          0  ...      177

[10 rows x 3 columns]

In [ ]:
df[df.text_len > 200].head(5)


sentiment  ... text_len
248973          0  ...      226
305242          0  ...      213
322655          0  ...      211
324570          0  ...      227
325649          0  ...      236

[5 rows x 3 columns]

In [ ]:
df[df.text_len >50].tail(10)


sentiment  ... text_len
1599985          4  ...      139
1599987          4  ...       73
1599988          4  ...      125
1599991          4  ...      133
1599994          4  ...      132
1599995          4  ...       56
1599996          4  ...       78
1599997          4  ...       57
1599998          4  ...       65
1599999          4  ...       62

[10 rows x 3 columns]

##### from the above dataframes we can see inside the text column that there are urls, @username mentions , HTML decoding error where we can see that the decoding is not text converted  like &quot etc, also there are many gibberish values that does not make sense. Also there are some words like won't haven't which we will change to would not and have not for the model to train better along with handling all the above things mentioned. 




  

##### Now we write a function to transform the tweets and solve above mentioned problems 



In [ ]:
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup


In [ ]:
tok = WordPunctTokenizer()
#handling urls and user mentions
combined_pat = r'|'.join((r'@[A-Za-z0-9_]+', r'https?://[^ ]+'))

negations_catalog = {"isn't":"is not", 
                     "aren't":"are not", 
                     "wasn't":"was not", 
                     "weren't":"were not",
                     "haven't":"have not",
                     "hasn't":"has not",
                     "hadn't":"had not",
                     "won't":"will not",
                     "wouldn't":"would not", 
                     "don't":"do not", 
                     "doesn't":"does not",
                     "didn't":"did not",
                     "can't":"can not",
                     "couldn't":"could not",
                     "shouldn't":"should not",
                     "mightn't":"might not",
                     "mustn't":"must not"}

neg_regex = re.compile(r'\b(' + '|'.join(negations_catalog.keys()) + r')\b')

def tweets_transformer(text):
  # handling decoding of HTML 
    soup = BeautifulSoup(text, 'lxml')
    get_soup_text = soup.get_text()
    
    try:
        bom_eliminator = get_soup_text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_eliminator = get_soup_text
        
    strip_text = re.sub(combined_pat, '', bom_eliminator)
    strip_text = re.sub(r'www.[^ ]+', '', strip_text)
    lower_case = strip_text.lower()
    # converting words from isn't to is not and likewise for all similar words inside the text. 
    neg_transform = neg_regex.sub(lambda x: negations_catalog[x.group()], lower_case)
    # handling hashtags and numbers 
    letters_filter = re.sub("[^a-zA-Z]", " ", neg_transform)
    words = [x for x  in tok.tokenize(letters_filter) if len(x) > 1]
    cleaned_text = (" ".join(words)).strip()
    return cleaned_text

In [ ]:
check_tweets = []
for t in df.text[:100]:
    check_tweets.append(tweets_transformer(t))

In [ ]:
pd.DataFrame(check_tweets).head(20)

0
0                                           awww that bummer you shoulda got david carr of third day to do it
1   is upset that he can not update his facebook by texting it and might cry as result school today also blah
2                                     dived many times for the ball managed to save the rest go out of bounds
3                                                              my whole body feels itchy and like its on fire
4                            no it not behaving at all mad why am here because can not see you all over there
5                                                                                          not the whole crew
6                                                                                                    need hug
7                                         hey long time no see yes rains bit only bit lol fine thanks how you
8                                                                                   nope they did not have it
9                                                                                                que me muera
10                                                                      spring break in plain city it snowing
11                                                                                    just re pierced my ears
12                                        could not bear to watch it and thought the ua loss was embarrassing
13                                               it it counts idk why did either you never talk to me anymore
14                 would ve been the first but did not have gun not really though zac snyder just doucheclown
15                                            wish got to watch it with you miss you and how was the premiere
16                 hollis death scene will hurt me severely to watch on film wry is directors cut not out now
17                                                                                        about to file taxes
18                                                      ahh ive always wanted to see rent love the soundtrack
19                                                oh dear were you drinking out of the forgotten table drinks

##### Now the text looks much better on which we can create our machine learning model in pyspark. 


